In [2]:
# set up earth engine
import ee

ee.Authenticate()

Enter verification code: 4/1AWtgzh5tGw8fzrH_pB8RG6v9eOzN5HzInttXEzRBjWdsbfp7jVuuC7t9qHU

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
# import prisons
prison_centroids = ee.FeatureCollection("projects/ee-ccmothes/assets/prison_centroids")


In [5]:
# define date range
startDate = "2012-01-01"
endDate = "2022-12-31"


In [6]:
# Define function to convert Kelvin to Celcius
def toCelciusDay(image):
  lst = image.select('LST_Day_1km').multiply(0.02).subtract(273.15)
  overwrite = True
  result = image.addBands(lst, ['LST_Day_1km'], overwrite)
  return result

In [7]:
# Quality mask; code adopted from https://spatialthoughts.com/2021/08/19/qa-bands-bitmasks-gee/
def bitwiseExtract(input, fromBit, toBit):
  maskSize = ee.Number(1).add(toBit).subtract(fromBit)
  mask = ee.Number(1).leftShift(maskSize).subtract(1)
  return input.rightShift(fromBit).bitwiseAnd(mask)


#Let's extract all pixels from the input image where
#Bits 0-1 <= 1 (LST produced of both good and other quality)
#Bits 2-3 = 0 (Good data quality)
#Bits 4-5 Ignore, any value is ok
#Bits 6-7 = 0 (Average LST error ≤ 1K)
def applyQaMask(image):
  lstDay = image.select('LST_Day_1km')
  qcDay = image.select('QC_Day')
  qaMask = bitwiseExtract(qcDay, 0, 1).lte(1)
  dataQualityMask = bitwiseExtract(qcDay, 2, 3).eq(0)
  lstErrorMask = bitwiseExtract(qcDay, 6, 7).lte(1)
  mask = qaMask.And(dataQualityMask).And(lstErrorMask)
  return lstDay.updateMask(mask)


In [8]:
#import MODIS
modisdata = ee.ImageCollection('MODIS/061/MYD11A1') \
  .filterDate(ee.Date(startDate),ee.Date(endDate)) \
  .filter(ee.Filter.calendarRange(6, 8,'month'))
  

#print(modisdata.first())

In [9]:
# Apply processing functions
lst_day_processed = modisdata.map(toCelciusDay).map(applyQaMask)

In [10]:
# Now calculate average summer day temperature across date range
summer_day_lst = lst_day_processed.select('LST_Day_1km').median()

In [38]:

# Extract values at prison centroids
sampled_points = summer_day_lst.sampleRegions(
  collection=prison_centroids,
  scale=1000
)                         
    

In [32]:
results = sampled_points.flatten()

In [33]:
print(results)


ee.FeatureCollection({
  "functionInvocationValue": {
    "functionName": "Collection.flatten",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Image.sampleRegions",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.loadTable",
                "arguments": {
                  "tableId": {
                    "constantValue": "projects/ee-ccmothes/assets/prison_centroids"
                  }
                }
              }
            },
            "image": {
              "functionInvocationValue": {
                "functionName": "reduce.median",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.map",
                      "arguments": {
                        "baseAlgorithm": {
                          "functionDefinition

In [35]:
# try using other method

#yearly = ee.ImageCollection('JRC/GSW1_0/YearlyHistory')

# function to map over the FeatureCollection
def mapfunc(feat):
  #get feature geometry
    geom = feat.geometry()
    # function to iterate over the yearly ImageCollection
    # the initial object for the iteration is the feature
    def addProp(img, f):
    # cast Feature
    newf = ee.Feature(f)
    #get date as string
    date = img.date().format()
    #extract the value (first) of 'waterClass' in the feature
    value = img.reduceRegion(ee.Reducer.first(), geom, 30).get('waterClass')
    #if the value is not null, set the values as a property of the feature. The name of the property will be the date
    return ee.Feature(ee.Algorithms.If(value,
                                       newf.set(date, ee.String(value)),
                                       newf.set(date, ee.String('No data'))))

  newfeat = ee.Feature(summer_day_lst.iterate(addProp, feat))
  return newfeat


newft = prison_centroids.map(mapfunc)

#Export
#Export.table.toDrive(newft,
#"export_Points",
#"export_Points",
#"export_Points");



{'type': 'Image',
 'bands': [{'id': 'LST_Day_1km',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -273.15,
    'max': 1037.5500000000002},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}